In [1]:
from utils.custom_document_reader import CustomDocumentReader

documents = CustomDocumentReader(
    input_dir="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/data",
    cache_dir="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/code/llamaIndex/.cache",
    config_path="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/code/llamaIndex/utils/config.json",
    remove_cache=False
)._load_data()

In [2]:
from llama_index.core.node_parser import SentenceSplitter

ss1 = SentenceSplitter(
    chunk_size=1024,
    include_metadata=True,
    include_prev_next_rel=True,
)

In [3]:
from llama_index.core.storage.docstore import SimpleDocumentStore
nodes = ss1.get_nodes_from_documents(documents)
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
docstore.persist(persist_path="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/code/llamaIndex/.cache/test.json")


In [8]:
from llama_index.core.node_parser import SimpleFileNodeParser

nodes = SimpleFileNodeParser.get_nodes_from_documents(documents=documents)

TypeError: NodeParser.get_nodes_from_documents() missing 1 required positional argument: 'documents'

In [9]:
for k, v in nodes[0].to_dict().items():
    print(f"key: {k}\nvalue: {v}\n\n")

key: id_
value: 9a552180-6da4-4e61-ab2c-48f113195f75


key: embedding
value: None


key: metadata
value: {'title': "THE ROLE OF FREE RADICALS IN THE PATHOPHYSIOLOGY OF PARKINSON'S DISEASE", 'authors': ['Sultana Banulescu'], 'sections': [[80, 1324], [1326, 5554], [5556, 11481], [11483, 14121], [14123, 16320], [16322, 18591], [18593, 25018], [25020, 28963], [28965, 31086], [31088, 33499], [33501, 36989], [36991, 54352]], 'file_name': '1_SHAKING_Palsy_The_Role_of_Free_Radical.pdf'}


key: excluded_embed_metadata_keys
value: []


key: excluded_llm_metadata_keys
value: []


key: relationships
value: {<NodeRelationship.SOURCE: '1'>: {'node_id': 'c1f72f5a-0fac-49f5-8a19-de4453a2ee44', 'node_type': <ObjectType.DOCUMENT: '4'>, 'metadata': {'title': "THE ROLE OF FREE RADICALS IN THE PATHOPHYSIOLOGY OF PARKINSON'S DISEASE", 'authors': ['Sultana Banulescu'], 'sections': [[80, 1324], [1326, 5554], [5556, 11481], [11483, 14121], [14123, 16320], [16322, 18591], [18593, 25018], [25020, 28963], [28965,

In [1]:
from utils.custom_parser import CustomHierarchicalNodeParser
hnp = CustomHierarchicalNodeParser.from_defaults()

In [3]:
hnp.get_nodes_from_documents(documents)

Doc ID: 85244a98-09e5-40ca-bb1c-ca37e37e41d6
Text: Title: THE ROLE OF FREE RADICALS IN THE PATHOPHYSIOLOGY OF
PARKINSON'S DISEASE  [1. Abstract] Parkinson's disease is a chronic
illness classified within the category of neurodegenative diseases
along with Alzheimer's disease, Huntington's disease, amyotrophic
lateral sclerosis, Friedreich ataxia and others. It consists of
progressive degeneratio...


NameError: name 'exit' is not defined